In [43]:
import os
import json
import pandas as pd
from pandas import json_normalize

def load_nested_json3(filepath):
    """Loads from dictionary containing a list of dictionaries nested json and flattens the data frame."""
    try:
        with open(filepath, 'r', encoding='utf-8') as f:  # Added encoding
            json_data = json.load(f)

        # Check for required keys before proceeding
        if 'status' not in json_data or 'data' not in json_data or 'results' not in json_data['data']:
            print(f"Warning: Missing key(s) in JSON file: {filepath}")
            return None  # Skip this file

        results = json_data['data']['results']  # access data by the key "results"
        df = pd.DataFrame(results)  # create a dataframe from the "results" value

        # --- Selective Extraction and Flattening ---
        # 1. Top-Level Columns (No flattening needed)
        top_level_columns = ['status', 'list_date', 'list_price', 'price_reduced_amount']
        df_top_level = df[top_level_columns].copy() if all(col in df.columns for col in top_level_columns) else pd.DataFrame()

        # 2. Description (Extract all keys)
        if 'description' in df.columns:
            df_description = pd.json_normalize(df['description'])
            df = pd.concat([df_top_level, df_description], axis=1)
        else:
            df = df_top_level

        # 3. Location (Extract state and city)
        if 'location' in df.columns:
              df_location = pd.json_normalize(df['location'])
              if 'address.state' in df_location.columns and 'address.city' in df_location.columns:
                  df_location = df_location[['address.state', 'address.city']]
                  df_location.columns = ['state', 'city']
                  df_top_level = pd.concat([df_top_level, df_location], axis=1)
              else:
                  print(f"Warning: 'address.state' or 'address.city' not found in location data in file: {filepath}")

        

        # 4. Branding (Extract name)
        if 'branding' in df.columns:
            df_branding = pd.json_normalize(df['branding'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else {}))
            if 'name' in df_branding.columns:
                df_branding = df_branding[['name']]
                df_branding.columns = ['branding_name']
                df = pd.concat([df, df_branding], axis=1)
            else:
                print(f"Warning: 'name' not found in branding data in file: {filepath}")

        # 5. Tags (Convert list to comma-separated string)
        if 'tags' in df.columns:
            df['tags'] = df['tags'].apply(lambda x: ', '.join(x) if isinstance(x, list) else '')

        return df

    except FileNotFoundError:
        print(f"Error: File not found at {filepath}")
        return None  # Return None if the file isn't found
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON in file {filepath}: {e}")
        return None  # Return None if JSON decoding fails
    except KeyError as e:
        print(f"Error: Key '{e}' not found in JSON file {filepath}")
        return None  # Return None if a key is missing
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None

def flatten_all_json_files(directory):
    all_dataframes = []

    # Iterate over all files in the directory
    for  filename in os.listdir(directory):
        if filename.endswith(".json"):
            filepath = os.path.join(directory, filename)
            flattened_df = load_nested_json3(filepath)
            if flattened_df is not None:
                all_dataframes.append(flattened_df)

            # Stop once the limit is reached
            #if len(all_dataframes) >= file_limit:
                #break

    # Concatenate all dataframes into a single dataframe
    if all_dataframes:
        combined_df = pd.concat(all_dataframes, ignore_index=True, sort=False)
        return combined_df
    else:
        print("No valid JSON files found.")
        return None

# Usage example:
directory = r"C:\Users\johnk\DS-midterm_project\data"  # Replace with the path to your directory containing JSON files
combined_df = flatten_all_json_files(directory)
if combined_df is not None:
    print(combined_df.head())
    # Save the combined dataframe to a CSV file if needed
    #combined_df.to_csv('flattened_data.csv', index=False)

  status                    list_date  list_price  price_reduced_amount  \
0   sold  2023-06-29T21:16:25.000000Z    554950.0               45000.0   
1   sold                         None         NaN                   NaN   
2   sold                         None         NaN                   NaN   
3   sold                         None         NaN                   NaN   
4   sold                         None         NaN                   NaN   

   year_built  baths_3qtr   sold_date  sold_price  baths_full  name  \
0      1963.0         NaN  2023-09-18         NaN         2.0  None   
1         NaN         NaN  2023-08-22         NaN         NaN  None   
2         NaN         NaN  2023-08-22         NaN         NaN  None   
3         NaN         NaN  2023-08-21         NaN         NaN  None   
4         NaN         NaN  2023-08-21         NaN         NaN  None   

   baths_half  lot_sqft    sqft  baths sub_type baths_1qtr  garage  stories  \
0         NaN   10454.0  1821.0    2.0     

C:\Users\johnk\AppData\Local\Temp\ipykernel_9336\2123354245.py:90: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_df = pd.concat(all_dataframes, ignore_index=True, sort=False)


In [48]:
loc_df = pd.read_csv(r"C:\Users\johnk\DS-midterm_project\location_data.csv")
loc_df.head()

,Unnamed: 0,city,county,lat,lon
0,0,Alaska,Juneau,58.36395,-134.59372
1,1,Alaska,Juneau,NaN,NaN
2,2,Alaska,Juneau,NaN,NaN
3,3,Alaska,Juneau,NaN,NaN
4,4,Alaska,Juneau,NaN,NaN


In [50]:
merged_df = pd.concat([combined_df, loc_df], axis =1)
merged_df.shape

(8159, 25)

In [56]:
merged_df

,status,list_date,list_price,price_reduced_amount,year_built,baths_3qtr,sold_date,sold_price,baths_full,baths_half,...,baths,sub_type,garage,stories,beds,type,city,county,lat,lon
0,sold,2023-06-29T21:16:25.000000Z,554950.0,45000.0,1963.0,NaN,2023-09-18,NaN,2.0,NaN,...,2.0,None,1.0,NaN,3.0,single_family,Alaska,Juneau,58.363950,-134.593720
1,sold,None,NaN,NaN,NaN,NaN,2023-08-22,NaN,NaN,NaN,...,NaN,None,NaN,NaN,NaN,None,Alaska,Juneau,NaN,NaN
2,sold,None,NaN,NaN,NaN,NaN,2023-08-22,NaN,NaN,NaN,...,NaN,None,NaN,NaN,NaN,None,Alaska,Juneau,NaN,NaN
3,sold,None,NaN,NaN,NaN,NaN,2023-08-21,NaN,NaN,NaN,...,NaN,None,NaN,NaN,NaN,None,Alaska,Juneau,NaN,NaN
4,sold,None,NaN,NaN,NaN,NaN,2023-08-21,NaN,NaN,NaN,...,NaN,None,NaN,NaN,NaN,None,Alaska,Juneau,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8154,sold,2023-04-05T19:28:04Z,98000.0,NaN,1910.0,NaN,2023-07-27,99000.0,1.0,NaN,...,1.0,None,1.0,2.0,3.0,single_family,West Virginia,Kanawha,38.341576,-81.644994
8155,sold,2023-05-28T02:25:28Z,49999.0,NaN,NaN,NaN,2023-07-27,29700.0,1.0,NaN,...,1.0,None,NaN,NaN,3.0,single_family,West Virginia,Kanawha,38.377371,-81.661662
8156,sold,2023-01-24T18:27:46Z,133000.0,NaN,NaN,NaN,2023-07-24,162250.0,1.0,NaN,...,1.0,None,NaN,NaN,3.0,single_family,West Virginia,Kanawha,38.338617,-81.659885
8157,sold,2023-03-24T12:27:42Z,75000.0,NaN,NaN,NaN,2023-07-24,63800.0,NaN,NaN,...,0.0,None,NaN,NaN,0.0,single_family,West Virginia,Kanawha,38.363038,-81.644214


In [53]:
merged_df = merged_df.drop(columns=['name',"baths_1qtr","Unnamed: 0"])

In [57]:
merged_df

,status,list_date,list_price,price_reduced_amount,year_built,baths_3qtr,sold_date,sold_price,baths_full,baths_half,...,baths,sub_type,garage,stories,beds,type,city,county,lat,lon
0,sold,2023-06-29T21:16:25.000000Z,554950.0,45000.0,1963.0,NaN,2023-09-18,NaN,2.0,NaN,...,2.0,None,1.0,NaN,3.0,single_family,Alaska,Juneau,58.363950,-134.593720
1,sold,None,NaN,NaN,NaN,NaN,2023-08-22,NaN,NaN,NaN,...,NaN,None,NaN,NaN,NaN,None,Alaska,Juneau,NaN,NaN
2,sold,None,NaN,NaN,NaN,NaN,2023-08-22,NaN,NaN,NaN,...,NaN,None,NaN,NaN,NaN,None,Alaska,Juneau,NaN,NaN
3,sold,None,NaN,NaN,NaN,NaN,2023-08-21,NaN,NaN,NaN,...,NaN,None,NaN,NaN,NaN,None,Alaska,Juneau,NaN,NaN
4,sold,None,NaN,NaN,NaN,NaN,2023-08-21,NaN,NaN,NaN,...,NaN,None,NaN,NaN,NaN,None,Alaska,Juneau,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8154,sold,2023-04-05T19:28:04Z,98000.0,NaN,1910.0,NaN,2023-07-27,99000.0,1.0,NaN,...,1.0,None,1.0,2.0,3.0,single_family,West Virginia,Kanawha,38.341576,-81.644994
8155,sold,2023-05-28T02:25:28Z,49999.0,NaN,NaN,NaN,2023-07-27,29700.0,1.0,NaN,...,1.0,None,NaN,NaN,3.0,single_family,West Virginia,Kanawha,38.377371,-81.661662
8156,sold,2023-01-24T18:27:46Z,133000.0,NaN,NaN,NaN,2023-07-24,162250.0,1.0,NaN,...,1.0,None,NaN,NaN,3.0,single_family,West Virginia,Kanawha,38.338617,-81.659885
8157,sold,2023-03-24T12:27:42Z,75000.0,NaN,NaN,NaN,2023-07-24,63800.0,NaN,NaN,...,0.0,None,NaN,NaN,0.0,single_family,West Virginia,Kanawha,38.363038,-81.644214


In [61]:
merged_df.to_csv(r"C:\Users\johnk\DS-midterm_project\merged_data.csv", index=False)
